In [ ]:
import routeur

LAMBDA_REQUETE = 1.8

C = 6

T_MAX = 1000

routeur.simul_fifo(LAMBDA_REQUETE, C, T_MAX)

[[0, 1],
 [0, 2],
 [0.2632553094146689, 3],
 [0.8333333333333334, 2],
 [1.183898635110703, 2],
 [1.3347223245205826, 3],
 [1.6383791258974914, 4],
 [1.6666666666666667, 3],
 [1.7741980521354912, 4],
 [1.8773235092489138, 5],
 [2.5, 4],
 [2.8063581525678956, 4],
 [3.3333333333333335, 3],
 [3.5531390594154297, 3],
 [3.8928335374565686, 3],
 [4.166666666666667, 2],
 [4.312791877305795, 3],
 [4.502798648349838, 4],
 [4.672186115831009, 5],
 [5.0, 4],
 [5.396539816948524, 5],
 [5.505994162290423, 6],
 [5.682702093239093, 7],
 [5.833333333333333, 6],
 [5.870900678550375, 6],
 [6.21826436044458, 6],
 [6.6663098415561866, 6],
 [6.666666666666666, 5],
 [6.889779812330248, 6],
 [7.499999999999999, 5],
 [7.610499338606591, 5],
 [7.9767102750707295, 5],
 [8.333333333333332, 4],
 [8.530889520293412, 5],
 [8.996759390644097, 6],
 [9.090499742116835, 7],
 [9.166666666666666, 6],
 [9.168246569447032, 6],
 [9.327379415571327, 7],
 [10.0, 6],
 [10.833333333333334, 5],
 [11.247328441427422, 5],
 [11.6666